# **Sentiment Analysis(Check)**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
# fasttext.util.download_model('ko', if_exists='ignore')   # FastText 모델 사용 시에만 필요
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

## **Read Data**

In [2]:
corp_list = ['samsung', 'hyundai', 'lg', 'sk', 'celltrion']

for i in range(len(corp_list)):
    globals()[corp_list[i]] = pd.read_csv('../../../../Final Data/{}. {}_score.csv'.format(i+1, corp_list[i]))
    globals()[corp_list[i] + '_maeil'] = globals()[corp_list[i]][globals()[corp_list[i]]['news']=='매일경제'].reset_index()
    globals()[corp_list[i] + '_asia'] = globals()[corp_list[i]][globals()[corp_list[i]]['news']=='아시아경제'].reset_index()

In [3]:
samsung_maeil[samsung_maeil.index==8400]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
8400,13079,삼성전자,5930,매일경제,2019111808,"[Hot-Line] ""옵트론텍, 3분기 실적 컨센서스 웃돌아""",http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n자료 제공 = 대신증권\n대신증권은 18일 옵트론텍에 대해 3분기 실적이...,2019-11-18,8,...,53500,7746554,-0.003724,-1,자료 제공 대신 증권 증권 옵트론텍 대해 실적 컨센서스 목표 주가 기존 상향 조정 ...,11,1,0.916667,1,183.333333


In [4]:
samsung_maeil[samsung_maeil['title']=="원유파생상품·급락장 베팅한 개미들은 `눈물`"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
10882,19524,삼성전자,5930,매일경제,2020060817,원유파생상품·급락장 베팅한 개미들은 `눈물`,http://news.mk.co.kr/newsRead.php?no=587231&ye...,◆ 新 머니무브 ② ◆\r\n\n\n 코스피가 연초 주가를 회복하고 2200선을 바...,2020-06-09,17,...,55500,23998831,0.010929,1,머니무브 코스피 연초 주가 주식 투자 나선 투자자 대부분 손절매 고민 투자자 개별 ...,3,3,0.5,1,100.0


In [5]:
samsung_maeil[samsung_maeil['title']=="[포토] 여름 오기전에…삼성 무풍에어컨공장 비지땀"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
5961,10640,삼성전자,5930,매일경제,2019050317,[포토] 여름 오기전에…삼성 무풍에어컨공장 비지땀,http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n 삼성전자가 여름 성수기를 앞두고 지난 3월부터 에어컨 생산라인을 풀가동...,2019-05-07,17,...,44850,12014907,-0.009934,-1,전자 여름 수기 지난 에어컨 생산 라인 가동 삼성 전자 광주 사업 직원 차림 생산 ...,0,0,0.5,1,101.0


In [6]:
samsung_maeil[samsung_maeil['title']=="외국인 더살까…FOMC·G2협상에 쏠린눈"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
5635,10314,삼성전자,5930,매일경제,2019040718,외국인 더살까…FOMC·G2협상에 쏠린눈,http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n 코스피에 대한 외국인 매수세가 이어지고 있다. 2분기 중 2300선으로...,2019-04-08,18,...,46650,8507909,-0.004269,-1,코스피 대한 외국인 전망 미국 중국 협상 낙관 흐름 보이 글로벌 경기 둔화 진정 국...,6,6,0.5,1,100.0


In [7]:
samsung_maeil[samsung_maeil['title']=='[Mobile World] "디지털 전환 출발선…하이브리드클라우드·AI가 무기"']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
10487,19129,삼성전자,5930,매일경제,2020051204,"[Mobile World] ""디지털 전환 출발선…하이브리드클라우드·AI가 무기""",http://news.mk.co.kr/newsRead.php?no=481305&ye...,\n\n\nIBM이 마이크로소프트(MS) 애저와 아마존웹서비스(AWS)가 주도하고 ...,2020-05-12,4,...,47900,23433590,-0.010331,-1,마이크로소프트 애저 아마존 웹서비스 클라우드 시장 격변 예고 아르 빈드 크리슈나 하...,2,1,0.666667,1,133.333333


In [8]:
samsung_maeil[samsung_maeil['title']=="[부음] 이특구씨(서울시립대 건축공학과 명예교수) 별세"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
7632,12311,삼성전자,5930,매일경제,2019091617,[부음] 이특구씨(서울시립대 건축공학과 명예교수) 별세,http://news.mk.co.kr/newsRead.php?year=2019&no...,"▲이특구씨(서울시립대 건축공학과 명예교수) 별세, 이방복(한국국제교류재단) 이민복씨...",2019-09-17,17,...,46900,7006280,-0.004246,-1,이특 구씨 서울 시립대 건축 학과 명예 교수 별세 이방 한국 국제 교류 재단 이민 ...,0,0,0.5,1,101.0


In [9]:
hyundai_maeil[hyundai_maeil['title']=="삼성전자 하이닉스 현대차 덕에 수출 3월도 두자릿수 증가 [종합]"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
6992,12709,현대차,5380,매일경제,2021032210,삼성전자 하이닉스 현대차 덕에 수출 3월도 두자릿수 증가 [종합],http://news.mk.co.kr/newsRead.php?no=269027&ye...,올 3월 들어서도 반도체와 승용차의 수출 호조세가 이어지고 있는 것으로 나타났다....,2021-03-22,10,...,229000,739527,-0.002179,-1,반도체 승용차 수출 호조 반도체 수출 개월 연속 자릿수 증가 기록 관세청 통관 기준...,9,1,0.9,1,180.0


In [10]:
hyundai_maeil[hyundai_maeil['title']=="더 똑똑해진 `신형 쏘나타`…카카오 AI 비서 탑재"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2577,4511,현대차,5380,매일경제,2019031809,더 똑똑해진 `신형 쏘나타`…카카오 AI 비서 탑재,http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n현대자동차는 21일 출시되는 신형 쏘나타에 카카오와 협력해 개발한 음성인...,2019-03-18,9,...,124000,383652,0.024793,1,자동차 출시 신형 쏘나타 카카오 협력 개발 음성인식 대화 비서 서비스 최초 탑재 사...,5,0,1.0,1,200.0


In [11]:
hyundai_maeil[hyundai_maeil['title']=="동학개미운동 이번엔 이겼다"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
5165,8908,현대차,5380,매일경제,2020060317,동학개미운동 이번엔 이겼다,http://news.mk.co.kr/newsRead.php?no=569723&ye...,◆ 글로벌 증시 동반 랠리 ◆\r연초부터 전례 없는 매수세로 일명 '동학개미운동'이...,2020-06-04,17,...,108500,2823041,0.0,0,글로벌 증시 동반 랠리 연초 전례 매수 세로 일명 동학 개미 운동 이란 유행어 탄생...,5,0,1.0,1,200.0


In [12]:
hyundai_maeil[hyundai_maeil['title']=='중고차시장 전면개방 요구 확산…교통연대 "소비자피해 방치말라"']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
6912,12629,현대차,5380,매일경제,2021030917,"중고차시장 전면개방 요구 확산…교통연대 ""소비자피해 방치말라""",http://news.mk.co.kr/newsRead.php?no=226188&ye...,국내 중고차 시장에 현대자동차와 기아 등 완성차 업계도 진입해야 한다는 소비자·시민...,2021-03-10,17,...,227500,901351,-0.017279,-1,국내 중고차 시장 자동차 기아 완성 업계 진입 소비자 시민단체 주장 확산 교통 자동...,4,7,0.363636,-1,72.727273


In [13]:
hyundai_maeil[hyundai_maeil['title']=="롯데 신동빈 80억 1위…LG 구광모, SK 최태원, 현대차 정의선 순, 상반기 연봉"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
7856,13573,현대차,5380,매일경제,2021081813,"롯데 신동빈 80억 1위…LG 구광모, SK 최태원, 현대차 정의선 순, 상반기 연봉",http://news.mk.co.kr/newsRead.php?no=799528&ye...,"\n\n\n(왼쪽부터) 이재용 삼성전자 부회장, 최태원 SK그룹 회장, 정의선 현대...",2021-08-18,13,...,212500,671920,-0.002347,-1,왼쪽 이재용 삼성 전자 부회장 최태원 그룹 회장 정의선 현대차 그룹 부회장 구광모 ...,0,1,0.0,-1,0.0


In [14]:
lg_maeil[lg_maeil['title']=="코스피, 기관 매수에 강보합 출발"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
793,793,LG화학,51910,매일경제,2018121709,"코스피, 기관 매수에 강보합 출발",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n 코스피가 기관 매수에 힘입어 강보합권에서 출발하고 있다.\r17일 오전...,2018-12-17,9,...,351500,133815,0.0,0,코스피 기관 매수 보합 출발 오전 현재 코스피 거래 대비 포인트 오른 기록 포인트 ...,1,5,0.166667,-1,33.333333


In [15]:
lg_maeil[lg_maeil['title']=="연초부터 뜨거운 韓 외화채권…1월에만 29억달러"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
1777,3466,LG화학,51910,매일경제,2020010517,연초부터 뜨거운 韓 외화채권…1월에만 29억달러,http://news.mk.co.kr/newsRead.php?no=12916&yea...,\n\n\n 국내 기업들이 경자년 벽두부터 외화채 발행에 연이어 나서고 있다. 원화...,2020-01-06,17,...,307000,138154,-0.012862,-1,국내 기업 경자년 벽두 화채 발행 원화 조달 비용 한국 채권 시장 대한 신뢰 때문 ...,3,3,0.5,1,100.0


In [16]:
lg_maeil[lg_maeil['title']=="코로나 확산에…인턴십 근무도 비대면 전환"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2420,4109,LG화학,51910,매일경제,2020081917,코로나 확산에…인턴십 근무도 비대면 전환,http://news.mk.co.kr/newsRead.php?no=854584&ye...,서울 등 수도권을 중심으로 코로나19 확산세가 심화되면서 국내 기업들이 사내 코로나...,2020-08-20,17,...,667000,841507,-0.023426,-1,서울 수도권 중심 코로나 확산 심화 국내 기업 사내 코로나 방역 총력 국내 출장 교...,0,1,0.0,-1,0.0


In [17]:
lg_maeil[lg_maeil['title']=="코스피, 개인 매수에 상승 출발…장중 2360선 회복"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2678,4367,LG화학,51910,매일경제,2020102309,"코스피, 개인 매수에 상승 출발…장중 2360선 회복",http://news.mk.co.kr/newsRead.php?no=1087191&y...,\n\n\n코스피가 개인의 매수세에 힘입어 상승 출발했다.\r23일 오전 9시16분...,2020-10-23,9,...,650000,405718,0.017214,1,코스피 개인 상승 출발 오전 현재 코스피 전일 대비 포인트 오른 기록 포인트 상승 ...,5,6,0.454545,-1,90.909091


In [18]:
lg_maeil[lg_maeil['title']=='[단독] "글로벌 배터리 인재 잡아라"…LG엔솔 임원진 美로 총출동']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
3403,5977,LG화학,51910,매일경제,2021070517,"[단독] ""글로벌 배터리 인재 잡아라""…LG엔솔 임원진 美로 총출동",http://news.mk.co.kr/newsRead.php?no=648922&ye...,\n\n\n 지난해 12월 1일 LG화학에서 분사한 LG에너지솔루션(사장 김종현·사...,2021-07-06,17,...,873000,156056,0.003448,1,지난해 화학 분사 에너지 솔루션 사장 김종현 사진 글로벌 인력 채용 김종현 사장 비...,4,0,1.0,1,200.0


In [19]:
sk_maeil[sk_maeil['title']=="코스피, 미중 무역협상 재개 기대감에 사흘째 오름세"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2113,3526,sk하이닉스,660,매일경제,2019090615,"코스피, 미중 무역협상 재개 기대감에 사흘째 오름세",http://news.mk.co.kr/newsRead.php?year=2019&no...,미국과 중국이 무역협상을 재개한다는 소식에 코스피가 3거래일 연속 오름세를 이어갔다...,2019-09-09,15,...,84300,2642174,0.029304,1,미국 중국 무역 협상 재개 소식 코스피 거래 연속 코스피 전일 대비 포인트 오른 거...,4,4,0.5,1,100.0


In [54]:
sk_maeil[sk_maeil['title']=='"SK 혁신기술력 널리 전파"…최재원 수석부회장의 주문']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
1266,2679,sk하이닉스,660,매일경제,2019012017,"""SK 혁신기술력 널리 전파""…최재원 수석부회장의 주문",http://news.mk.co.kr/newsRead.php?year=2019&no...,"\n\n\n ""내년에는 전시 공간을 더 확대하고 더 많은 SK의 혁신 기술을 공개하...",2019-01-21,17,...,67000,3977425,0.037152,1,내년 전시 공간 확대 혁신 기술 공개 최근 미국 라스베이거스 세계 최대 가전 정보통...,3,0,1.0,1,200.0


In [21]:
sk_maeil[sk_maeil['title']=="매경이 전하는 세상의 지식 (매-세-지, 6월 10일)"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
4071,8003,sk하이닉스,660,매일경제,2021061007,"매경이 전하는 세상의 지식 (매-세-지, 6월 10일)",http://premium.mk.co.kr/view.php?no=30306,"매경이 전하는 세상의 지식 (매-세-지, 6월 10일) \r1. 앞으로 투기과열지구...",2021-06-10,7,...,123000,4533076,0.004082,1,매경 전하 세상 지식 투기 지구 재건축 조합원 지위 안전 진단 통과 아파트 국토교통...,3,1,0.75,1,150.0


In [22]:
sk_maeil[sk_maeil['title']=='[컨콜] SKT "SKT배당 SK하이닉스 배당과 연계하기 어려워"']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
140,140,sk하이닉스,660,매일경제,2018020513,"[컨콜] SKT ""SKT배당 SK하이닉스 배당과 연계하기 어려워""",http://news.mk.co.kr/newsRead.php?year=2018&no...,유영상 SK텔레콤 최고재무책임자(CFO)는 5일 지난해 4분기 실적 발표 후 진행한...,2018-02-05,13,...,70200,4058439,-0.027701,-1,유영상 텔레콤 최고재 무책임 지난해 실적 발표 진행 컨퍼런스 배당 대한 텔레콤 입장...,1,0,1.0,1,200.0


In [23]:
sk_maeil[sk_maeil['title']=="SK하이닉스, 중국 투자 박차…현지법인에 3.3조 자금대여"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2790,5406,sk하이닉스,660,매일경제,2020040221,"SK하이닉스, 중국 투자 박차…현지법인에 3.3조 자금대여",http://news.mk.co.kr/newsRead.php?no=346636&ye...,SK하이닉스가 코로나19 확산에도 메모리 반도체 미세공정 전환을 위한 대규모 투자를...,2020-04-03,21,...,79600,3755301,-0.005,-1,하이닉스 코로나 확산 메모리 반도체 미세 공정 전환 대규모 투자 단행 하이닉스 자사...,1,0,1.0,1,200.0


In [24]:
celltrion_maeil[celltrion_maeil['title']=='[Hot-Line] "아이큐어, 도네페질 치매 패치제 임상3상 결과 발표 임박"']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2292,4451,셀트리온,68270,매일경제,2020071508,"[Hot-Line] ""아이큐어, 도네페질 치매 패치제 임상3상 결과 발표 임박""",http://news.mk.co.kr/newsRead.php?no=722614&ye...,\n\n\n자료 제공 = SK증권\nSK증권은 15일 아이큐어에 대해 도네페질 치매...,2020-07-15,8,...,310314,972746,-0.024691,-1,자료 제공 증권 증권 아이큐 대해 도네페질 치매 패치 상의 결과 발표 임박 목표 주...,4,0,1.0,1,200.0


In [25]:
celltrion_maeil[celltrion_maeil['title']=="코스피 1% 상승 마감…1700 문턱 넘어"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
1983,4142,셀트리온,68270,매일경제,2020032716,코스피 1% 상승 마감…1700 문턱 넘어,http://news.mk.co.kr/newsRead.php?no=320497&ye...,\n\n\n 코스피가 1%대 상승세를 보이면서 1700선 고지를 다시 회복했다.\r...,2020-03-30,16,...,181671,937759,0.005435,1,코스피 상승세 고지 다시 코스피 전일 대비 포인트 상승 오른 출발 지수 글로벌 증시...,4,7,0.363636,-1,72.727273
1984,4143,셀트리온,68270,매일경제,2020032716,코스피 1% 상승 마감…1700 문턱 넘어,http://news.mk.co.kr/newsRead.php?no=320497&ye...,\n\n\n 코스피가 1%대 상승세를 보이면서 1700선 고지를 다시 회복했다.\r...,2020-03-30,16,...,181671,937759,0.005435,1,코스피 상승세 고지 다시 코스피 전일 대비 포인트 상승 오른 출발 지수 글로벌 증시...,4,7,0.363636,-1,72.727273


In [26]:
celltrion_maeil[celltrion_maeil['title']=="작년 1000억 넘게 R&D 투자한 제약사 7곳…셀트리온, 압도적 1위"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
1166,2470,셀트리온,68270,매일경제,2019040308,"작년 1000억 넘게 R&D 투자한 제약사 7곳…셀트리온, 압도적 1위",http://news.mk.co.kr/newsRead.php?year=2019&no...,지난해 국내 제약·바이오업계에서 1000억원 이상의 연구·개발(R&D;) 비용을 쓴...,2019-04-03,8,...,181494,503210,0.018416,1,지난해 국내 제약 바이오 업계 이상 연구 개발 비용 회사 모두 금융감독원 자공시 시...,0,0,0.5,1,101.0


In [27]:
celltrion_maeil[celltrion_maeil['title']=="대형업체 전유물 CMO 시장, 중소바이오社 잇단 진출 왜?"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
1611,2915,셀트리온,68270,매일경제,2019102017,"대형업체 전유물 CMO 시장, 중소바이오社 잇단 진출 왜?",http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n 중소 바이오 기업들이 대거 의약품 위탁생산(CMO)에 뛰어들고 있다. ...,2019-10-21,17,...,170708,380996,0.002755,1,중소 바이오 기업 대거 의약품 위탁 생산 사업 대규모 시설 자본 그동안 바이오 직스...,7,1,0.875,1,175.0


In [28]:
celltrion_maeil[celltrion_maeil['title']=="코스피, 미중 무역협상 앞두고 `흔들`…기관·외국인 매도"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
349,349,셀트리온,68270,매일경제,2018050416,"코스피, 미중 무역협상 앞두고 `흔들`…기관·외국인 매도",http://news.mk.co.kr/newsRead.php?year=2018&no...,코스피가 미중 무역협상을 앞두고 큰 폭으로 흔들렸다. 글로벌 경제에 대한 우려가 매...,2018-05-08,16,...,217473,1422120,-0.056008,-1,코스피 무역 협상 글로벌 경제 대한 우려 매도 심리 자극 지수 하락 코스피 거래 대...,1,10,0.090909,-1,18.181818


In [29]:
samsung_asia[samsung_asia['title']=="외국인, 3주만에 '팔자' 전환…SK그룹株는 사들여"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
13684,27947,삼성전자,5930,아시아경제,2021022109,"외국인, 3주만에 '팔자' 전환…SK그룹株는 사들여",https://view.asiae.co.kr/article/2021022109321...,[아시아경제 송화정 기자]외국인이 국내 증시에서 3주 만에 순매도로 돌아섰다.21일...,2021-02-22,9,...,82200,25419886,-0.004843,-1,아시아 경제 송화 기자 외국인 국내 증시 도로 한국 거래소 외국인 지난 동안 국내 ...,9,4,0.692308,1,138.461538


In [30]:
samsung_asia[samsung_asia['title']=='[팩트체크] 靑 "문 대통령과 이재용 7번 만났다고 하는 건 과도한 집계"라는데…']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
6130,14988,삼성전자,5930,아시아경제,2019050111,"[팩트체크] 靑 ""문 대통령과 이재용 7번 만났다고 하는 건 과도한 집계""라는데…",http://view.asiae.co.kr/news/view.htm?idxno=20...,문재인 대통령이 30일 오후 시스템 반도체 비전 선포식이 열린 삼성전자 화성캠퍼스 ...,2019-05-02,11,...,45900,8625126,0.001091,1,문재인 대통령 오후 시스템 반도체 선포 식이 삼성 전자 화성 캠퍼스 부품 구동 발언...,0,0,0.5,1,101.0


In [31]:
samsung_asia[samsung_asia['title']=="국내 증시, 고점 부담·美기술주 하락 여파에 동반 약세"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
14747,29010,삼성전자,5930,아시아경제,2021051109,"국내 증시, 고점 부담·美기술주 하락 여파에 동반 약세",https://view.asiae.co.kr/article/2021051109383...,[이미지출처=연합뉴스][아시아경제 박지환 기자] 국내 증시가 장 초반 전날 지수 상...,2021-05-11,9,...,81200,28996680,-0.024038,-1,이미지 출처 연합뉴스 아시아 경제 박지환 기자 국내 증시 초반 전날 지수 상승 대한...,6,13,0.315789,-1,63.157895


In [32]:
samsung_asia[samsung_asia['title']=="전자랜드 서산점, '파워센터'로 리뉴얼 오픈"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
7002,15860,삼성전자,5930,아시아경제,2019072508,"전자랜드 서산점, '파워센터'로 리뉴얼 오픈",https://view.asiae.co.kr/article/2019072508120...,[아시아경제 성기호 기자] 전자랜드가 전북 전주시 완산구 서신동에 위치한 서신점을 ...,2019-07-25,8,...,47200,8408838,0.017241,1,아시아 경제 기호 기자 전자랜드 전북 전주시 완산구 서신동 위치 파워 센터 리뉴얼 ...,4,0,1.0,1,200.0


In [33]:
samsung_asia[samsung_asia['title']=="[평양회담]특별수행단, 北김영남 접견…경제인, 리용남 부총리 접견"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
3406,7770,삼성전자,5930,아시아경제,2018091815,"[평양회담]특별수행단, 北김영남 접견…경제인, 리용남 부총리 접견",http://view.asiae.co.kr/news/view.htm?idxno=20...,"정당 대표단, 北최고인민회의 부의장 접견…시민사회는 조선사회민주당과 만남 예정 [이...",2018-09-19,15,...,46150,9367454,0.014286,1,정당 대표 최고 인민 회의 부의 접견 시민사회 조선 사회 민주당 만남 예정 이미지 ...,3,0,1.0,1,200.0


In [34]:
hyundai_asia[hyundai_asia['title']=="[포토] 현대차 '넥쏘' 내부 공개"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
533,2637,현대차,5380,아시아경제,2018041213,[포토] 현대차 '넥쏘' 내부 공개,http://view.asiae.co.kr/news/view.htm?idxno=20...,12일 서울 강남구 코엑스에서 열린 'EV 트렌드 코리아 2018'에서 현대 수소전...,2018-04-12,13,...,152000,297375,0.013333,1,서울 강남구 코엑스 트렌드 코리아 현대 수소 전기차 넥쏘 공개 강진 기자 카카오 카...,1,0,1.0,1,200.0


In [35]:
hyundai_asia[hyundai_asia['title']=='이광국 부사장 "팰리세이드, 폭발적 반응에 판매 목표 상향"(종합)']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
1836,3940,현대차,5380,아시아경제,2018121112,"이광국 부사장 ""팰리세이드, 폭발적 반응에 판매 목표 상향""(종합)",http://view.asiae.co.kr/news/view.htm?idxno=20...,"팰리세이드, 사전계약 일평균 2563대로 산타페보다 1000대 많아이광국 국내영업 ...",2018-12-11,12,...,111500,372363,0.00905,1,팰리세이드 사전 계약 평균 대로 산타페 이광 국내 영업 본부장 생산 공급 고려 판매...,9,0,1.0,1,200.0


In [36]:
hyundai_asia[hyundai_asia['title']=="끝내 3000 밑으로…外人·기관 매도세에 내려 앉은 코스피"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
6320,14413,현대차,5380,아시아경제,2021032416,끝내 3000 밑으로…外人·기관 매도세에 내려 앉은 코스피,https://view.asiae.co.kr/article/2021032416055...,코스닥은 0.8% 올라…950선으로 회복 [이미지출처=연합뉴스][아시아경제 이민우 ...,2021-03-25,16,...,218000,1087519,-0.013575,-1,코스닥 회복 이미지 출처 연합뉴스 아시아 경제 이민우 기자 오전 코스피 끝내 하락 ...,3,4,0.428571,-1,85.714286


In [37]:
hyundai_asia[hyundai_asia['title']=="현대차 정몽구 재단, '산의 날' 유공 대통령 표창 수상"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
1483,3587,현대차,5380,아시아경제,2018101815,"현대차 정몽구 재단, '산의 날' 유공 대통령 표창 수상",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 우수연 기자] 현대차 정몽구 재단이 제 1...,2018-10-19,15,...,115500,493067,-0.00431,-1,아시아 경제 우수 기자 현대차 정몽구 재단 유공 대통령 표창 유공 대통령 표창 산림...,2,0,1.0,1,200.0


In [38]:
hyundai_asia[hyundai_asia['title']=='"유럽 수소상용차 시장 공략"…현대차, 스위스 수소에너지 기업과 합작법인 설립']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2559,6756,현대차,5380,아시아경제,2019041515,"""유럽 수소상용차 시장 공략""…현대차, 스위스 수소에너지 기업과 합작법인 설립",http://view.asiae.co.kr/news/view.htm?idxno=20...,[아시아경제 우수연 기자] 현대자동차가 스위스 수소 에너지기업 'H2 Energy(...,2019-04-16,15,...,128500,753845,0.023904,1,아시아 경제 우수 기자 자동차 스위스 수소 에너지 기업 이하 합작 법인 설립 유럽 ...,7,0,1.0,1,200.0


In [39]:
lg_asia[lg_asia['title']=="오후 코스피, 외국인·기관 순매수 속 3220선 등락"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2996,6552,LG화학,51910,아시아경제,2021042012,"오후 코스피, 외국인·기관 순매수 속 3220선 등락",https://view.asiae.co.kr/article/2021042012575...,"20일 서울 중구 을지로 하나은행 딜링룸 전광판에 코스피 지수, 원·달러 환율, 코...",2021-04-20,12,...,893000,232900,0.013621,1,서울 중구 을지로 하나은행 딜링룸 전광판 코스피 지수 달러 환율 코스닥 지수 호남 ...,3,2,0.6,1,120.0


In [40]:
lg_asia[lg_asia['title']=="동반성장위원회-LG화학, ‘협력사 ESG 지원사업’ 협약 체결"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
3411,6967,LG화학,51910,아시아경제,2021082410,"동반성장위원회-LG화학, ‘협력사 ESG 지원사업’ 협약 체결",https://view.asiae.co.kr/article/2021082410450...,교육·역량진단·컨설팅 등 협력사 ESG 종합지원 [아시아경제 김희윤 기자] 동반성...,2021-08-24,10,...,787000,895863,-0.013784,-1,교육 역량 진단 컨설팅 협력 종합 지원 아시아 경제 김희윤 기자 동반성 위원회 위원...,3,0,1.0,1,200.0


In [41]:
lg_asia[lg_asia['title']=="기관매수 지속…코스피 1%대 상승 유지"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
3092,6648,LG화학,51910,아시아경제,2021051815,기관매수 지속…코스피 1%대 상승 유지,https://view.asiae.co.kr/article/2021051814010...,코스닥도 970선 넘봐 18일 오전 서울 중구 명동 하나은행 본점 딜링룸에서 직원들...,2021-05-20,15,...,903000,318001,0.024972,1,코스닥 오전 서울 중구 명동 하나은행 본점 딜링룸 직원 근무 이미지 출처 연합뉴스 ...,3,2,0.6,1,120.0


In [42]:
lg_asia[lg_asia['title']=="[단독]에이프로, LG엔솔-GM 배터리공장에 1천억 규모 장비 공급 추진"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2992,6548,LG화학,51910,아시아경제,2021041915,"[단독]에이프로, LG엔솔-GM 배터리공장에 1천억 규모 장비 공급 추진",https://view.asiae.co.kr/article/2021041910451...,합작 배터리 1공장 15GWh가량 수주 진행…1.5GWh당 100억 규모총 계약 규...,2021-04-20,15,...,893000,232900,0.013621,1,합작 배터리 공장 가량 수주 진행 규모 계약 규모 추정 에이 프로 지난해 매출 이상...,47,0,1.0,1,200.0


In [43]:
lg_asia[lg_asia['title']=="[증권사 주간 추천 종목]SK증권"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
527,1348,LG화학,51910,아시아경제,2018081109,[증권사 주간 추천 종목]SK증권,http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t◆\r\t\t\t\t\t\t\t\tLG화학LG화학051...,2018-08-13,9,...,368000,216645,-0.018667,-1,화학 화학 코스피 증권 정보 현재 전일 대비 거래 전일 장마 관련 기사 개인 도세 ...,4,2,0.666667,1,133.333333
538,1359,LG화학,51910,아시아경제,2018081809,[증권사 주간 추천 종목]SK증권,http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t◆\r\t\t\t\t\t\t\t\tLG화학LG화학051...,2018-08-20,9,...,366000,150033,0.001368,1,화학 화학 코스피 증권 정보 현재 전일 대비 거래 전일 장마 관련 기사 개인 도세 ...,4,3,0.571429,1,114.285714


In [44]:
sk_asia[sk_asia['title']=="코스피, 상승 출발 후 하락 반전…2400선 붕괴(종합)"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
256,1446,sk하이닉스,660,아시아경제,2018030509,"코스피, 상승 출발 후 하락 반전…2400선 붕괴(종합)",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 조강욱 기자] 코스피 2400선이 다시 무...,2018-03-05,9,...,78300,3415061,0.011628,1,아시아 경제 조강 기자 코스피 다시 외국인 기관 도세 낙폭 모습 오전 현재 코스피 ...,3,4,0.428571,-1,85.714286


In [45]:
sk_asia[sk_asia['title']=="반도체 부품 수요 확대 추세…원익QnC 살아날까"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
4249,8614,sk하이닉스,660,아시아경제,2021032108,반도체 부품 수요 확대 추세…원익QnC 살아날까,https://view.asiae.co.kr/article/2021032019470...,"미국 한파에 오스틴 반도체 제조공장 운영 중단…""상반기 실적 호재"" [아시아경제 공...",2021-03-22,8,...,138000,2727719,0.0,0,미국 한파 오스틴 반도체 제조 공장 운영 중단 상반기 실적 호재 아시아 경제 공병 ...,4,4,0.5,1,100.0


In [46]:
sk_asia[sk_asia['title']=="코스피, FOMC까지 관망심리 커져…2050선까지 후퇴"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2150,4646,sk하이닉스,660,아시아경제,2019072909,"코스피, FOMC까지 관망심리 커져…2050선까지 후퇴",https://view.asiae.co.kr/article/2019072909055...,[아시아경제 박형수 기자] 코스피지수가 나흘 연속으로 하락하고 있다. 뉴욕증시에서 ...,2019-07-29,9,...,77000,2616782,-0.035088,-1,아시아 경제 박형수 기자 코스피지수 나흘 연속 하락 뉴욕증시 스탠더드 앤드 어스 지...,5,3,0.625,1,125.0


In [47]:
sk_asia[sk_asia['title']=="미국 금리인하 기대감…코스피 2090선 '회복'"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2121,4617,sk하이닉스,660,아시아경제,2019071916,미국 금리인하 기대감…코스피 2090선 '회복',https://view.asiae.co.kr/article/2019071916305...,[아시아경제 유현석 기자] 코스피와 코스닥이 미국의 기준금리 인하 기대감에 1% 넘...,2019-07-22,16,...,78400,3213786,0.020833,1,아시아 경제 유현 기자 코스피 코스닥 미국 기준금리 인하 대감 상승 코스피 거래 대...,7,3,0.7,1,140.0


In [48]:
sk_asia[sk_asia['title']=="[속수무책 기술유출]美 첨단산업 견제에 궁지 몰린 中…韓 기술 호시탐탐"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
4489,8854,sk하이닉스,660,아시아경제,2021050311,[속수무책 기술유출]美 첨단산업 견제에 궁지 몰린 中…韓 기술 호시탐탐,https://view.asiae.co.kr/article/2021050311172...,"반도체·2차전지 등 핵심산업, 中서 '기술 빼내가기' 타깃 돼당국, 한달에 한건꼴 ...",2021-05-03,11,...,131500,4085398,0.027344,1,반도체 핵심 산업 기술 타깃 당국 건꼴 적발 경찰청 오늘 특별 단속 세종 아시아 경...,4,11,0.266667,-1,53.333333


In [49]:
celltrion_asia[celltrion_asia['title']=="코스피, 11일 연속 상승…2100 넘어설까"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
1930,3708,셀트리온,68270,아시아경제,2019092210,"코스피, 11일 연속 상승…2100 넘어설까",https://view.asiae.co.kr/article/2019092210445...,"썝蹂몃낫湲 븘씠肄[아시아경제 고형광 기자] '126포인트, 6.4% 점프'. 코스피...",2019-09-23,10,...,160390,884762,-0.025642,-1,몃낫 븘씠 아시아 경제 형광 기자 포인트 점프 코스피지수 거래 연속 상승 어가 단숨...,3,2,0.6,1,120.0


In [50]:
celltrion_asia[celltrion_asia['title']=="다음 주 코스피 2490~2550 전망…주요국 통화정책 회의에 주목"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
82,1016,셀트리온,68270,아시아경제,2018012006,다음 주 코스피 2490~2550 전망…주요국 통화정책 회의에 주목,http://www.asiae.co.kr/news/view.htm?idxno=201...,지난해 4Q 실적 발표 시즌 본격 돌입…원·달러 환율은 1060~1085원 \r\t...,2018-01-22,6,...,258573,2778135,-0.025018,-1,지난해 실적 발표 시즌 본격 돌입 달러 환율 아시아 경제 임철영 기자 투자 증권 다...,6,4,0.6,1,120.0


In [51]:
celltrion_asia[celltrion_asia['title']=="코스피, 개인 순매수 나서며 상승 출발 … 코스닥 2%↑"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2797,5543,셀트리온,68270,아시아경제,2020081909,"코스피, 개인 순매수 나서며 상승 출발 … 코스닥 2%↑",https://view.asiae.co.kr/article/2020081909333...,[이미지출처=연합뉴스]썝蹂몃낫湲 븘씠肄[아시아경제 구은모 기자] 코스피와 코스닥 지...,2020-08-19,9,...,298530,880282,0.0,0,이미지 출처 연합뉴스 몃낫 븘씠 아시아 경제 기자 코스피 코스닥 지수 전날 부진 상...,6,7,0.461538,-1,92.307692


In [52]:
celltrion_asia[celltrion_asia['title']=="코스피, 글로벌 경기 부양 기대감에 1700선 회복… 코스닥 500선 복귀"]

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
2344,5090,셀트리온,68270,아시아경제,2020032515,"코스피, 글로벌 경기 부양 기대감에 1700선 회복… 코스닥 500선 복귀",https://view.asiae.co.kr/article/2020032515573...,[이미지출처=연합뉴스]썝蹂몃낫湲 븘씠肄[아시아경제 구은모 기자] 각국의 경기 부양을...,2020-03-26,15,...,178234,2441069,-0.008197,-1,이미지 출처 연합뉴스 몃낫 븘씠 아시아 경제 기자 각국 경기 부양 정책 대응 본격 ...,1,10,0.090909,-1,18.181818


In [53]:
celltrion_asia[celltrion_asia['title']=='''[클릭 e종목]"마이크로디지탈, 코로나 백신 생산 필수품 '세포배양백' 제조"''']

,index,st_n,st_cd,news,datetime,title,url,text,date,time,...,Close,Volume,Change,UpDown,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
3700,6921,셀트리온,68270,아시아경제,2021052409,"[클릭 e종목]""마이크로디지탈, 코로나 백신 생산 필수품 '세포배양백' 제조""",https://view.asiae.co.kr/article/2021052409032...,[아시아경제 유현석 기자] 리서치알음은 \r\t\t\t\t\t\t\t\t마이크로디지...,2021-05-24,9,...,262000,454113,-0.024209,-1,아시아 경제 유현 기자 리서치 알음 마이크로 디지탈 마이크로 디지탈 코스닥 증권 정...,3,1,0.75,1,150.0
